# Sentinel Hub Batch Processing

**Sentinel Hub Batch Processing** takes the geometry of a large area and divides it according to a specified tile grid. Next, it executes processing requests for each tile in the grid and stores results to a given location at AWS S3 storage. All this is efficiently executed on the server-side. Because of the optimized performance, it is significantly faster than running the same process locally. 

More information about batch processing is available at Sentinel Hub documentation pages:

- [How Batch API works](https://docs.sentinel-hub.com/api/latest/api/batch/)
- [Batch API service description](https://docs.sentinel-hub.com/api/latest/reference/#tag/batch_process)


The tutorial will show a standard process of using Batch Processing with `sentinelhub-py`. The process can be divided into:

1. Define and create a batch request
2. Analyse a batch request before it is executed
3. Run a batch requests job and check the outcome


In [ ]:
%matplotlib inline

import datetime as dt
import os

import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

from sentinelhub import (
    CRS,
    DataCollection,
    Geometry,
    MimeType,
    SentinelHubBatch,
    SentinelHubRequest,
    SHConfig,
    bbox_to_dimensions,
    monitor_batch_job,
)

## 1. Create a batch request

To create a batch request we need to do the following:

- Define a Process API request which we would like to execute on a large area.
- Select a tiling grid which will define how our area will be split into smaller tiles.
- Set up an S3 bucket where results will be saved.


### 1.1 Define a Process API request

In [ ]:
config = SHConfig()

if config.sh_client_id == "" or config.sh_client_secret == "":
    print("Warning! To use Sentinel Hub Process API, please provide the credentials (client ID and client secret).")

For our area of interest, we'll take an area of crop fields in California.

In [ ]:
SHAPE = """{
"type": "FeatureCollection",
"name": "california_crop_fields",
"crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },
"features": [
{ "type": "Feature", "properties": { "name": "Entire area" }, "geometry": { "type": "MultiPolygon", "coordinates": [ [ [ [ -119.2089, 36.1383 ], [ -119.1323, 36.0980 ], [ -119.1282, 35.9869 ], [ -119.2264, 35.9532 ], [ -119.3263, 35.9822 ], [ -119.3488, 36.0899 ], [ -119.2089, 36.1383 ] ] ] ] } },
{ "type": "Feature", "properties": { "name": "Test sub-area" }, "geometry": { "type": "MultiPolygon", "coordinates": [ [ [ [ -119.2778, 36.0806 ], [ -119.2776, 36.04 ], [ -119.2135, 36.04 ], [ -119.21409, 36.0806 ], [ -119.2778, 36.0806 ] ] ] ] } }
]
}"""
area_gdf = gpd.read_file(SHAPE)

# Geometry of an entire area
full_geometry = Geometry(area_gdf.geometry.values[0], crs=CRS.WGS84)
# Bounding box of a test sub-area
test_bbox = Geometry(area_gdf.geometry.values[1], crs=CRS.WGS84).bbox

area_gdf.plot(column="name");

We need to define function for plotting RGB images.

In [ ]:
from __future__ import annotations

from typing import Any

import matplotlib.pyplot as plt
import numpy as np


def plot_image(
    image: np.ndarray, factor: float = 1.0, clip_range: tuple[float, float] | None = None, **kwargs: Any
) -> None:
    """Utility function for plotting RGB images."""
    _, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 15))
    if clip_range is not None:
        ax.imshow(np.clip(image * factor, *clip_range), **kwargs)
    else:
        ax.imshow(image * factor, **kwargs)
    ax.set_xticks([])
    ax.set_yticks([])

Let's check a true-color satellite image of the entire area:

In [ ]:
evalscript_true_color = """
    //VERSION=3
    function setup() {
        return {
            input: [{
                bands: ["B02", "B03", "B04"]
            }],
            output: {
                bands: 3
            }
        };
    }
    function evaluatePixel(sample) {
        return [sample.B04, sample.B03, sample.B02];
    }
"""

request = SentinelHubRequest(
    evalscript=evalscript_true_color,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L2A,
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
    geometry=full_geometry,
    size=(512, 512),
    config=config,
)

image = request.get_data()[0]

plot_image(image, factor=3.5 / 255, clip_range=(0, 1))

Next, let's define an evalscript and time range. To better demonstrate the power of batch processing we'll take an evalscript that returns a temporally-interpolated stack NDVI values.

In the following cell parameters `evalscript` and `time_interval` are both defined for the same time interval. If you decide to change the time interval you have to change it both in the cell and in the evalscript code.


In [ ]:
evalscript = """
//VERSION=3

// Calculate number of bands needed for all intervals
// Initialize dates and interval
// Beware: in JS months are 0 indexed
var start_date = new Date(2023, 6, 1, 0, 0, 0);
var end_date = new Date(2023, 6, 30, 0, 0, 0);
var sampled_dates = sample_timestamps(start_date, end_date, 7, 'day').map(d => withoutTime(d));
var nb_bands = sampled_dates.length;
var n_valid = 0;
var n_all = 0;

function interval_search(x, arr) {
  let start_idx = 0,  end_idx = arr.length - 2;

  // Iterate while start not meets end
  while (start_idx <= end_idx) {
    // Find the mid index
    let mid_idx = (start_idx + end_idx) >> 1;

    // If element is present at mid, return True
    if (arr[mid_idx] <= x && x < arr[mid_idx + 1]) {
      return mid_idx;
    }
    // Else look in left or right half accordingly
    else if (arr[mid_idx + 1] <= x) start_idx = mid_idx + 1;
    else end_idx = mid_idx - 1;
  }
  if (x == arr[arr.length-1]){
    return arr.length-2;
  }
  return undefined;
}

function linearInterpolation(x, x0, y0, x1, y1, no_data_value=NaN) {
  if (x < x0 || x > x1) {
    return no_data_value;
  }
  var a = (y1 - y0) / (x1 - x0);
  var b = -a * x0 + y0;
  return a * x + b;
}

function lininterp(x_arr, xp_arr, fp_arr, no_data_value=NaN) {
  results = [];
  data_mask = [];
  xp_arr_idx = 0;
  for (var i=0; i<x_arr.length; i++) {
    var x = x_arr[i];
    n_all+=1;
    interval = interval_search(x, xp_arr);
    if (interval === undefined) {
      data_mask.push(0);
      results.push(no_data_value);
      continue;
    }
    data_mask.push(1);
    n_valid+=1;
    results.push(
      linearInterpolation(
        x,
        xp_arr[interval],
        fp_arr[interval],
        xp_arr[interval+1],
        fp_arr[interval+1],
        no_data_value
      )
    );
  }
  return [results, data_mask];
}

function interpolated_index(index_a, index_b) {
  // Calculates the index for all bands in array
  var index_data = [];
  for (var i = 0; i < index_a.length; i++){
     // UINT index returned
     let ind = (index_a[i] - index_b[i]) / (index_a[i] + index_b[i]);
     index_data.push(ind * 10000 + 10000);
  }
  return index_data
}

function increase(original_date, period, period_unit) {
    date = new Date(original_date)
    switch (period_unit) {
        case 'millisecond':
            return new Date(date.setMilliseconds(date.getMilliseconds()+period));
        case 'second':
            return new Date(date.setSeconds(date.getSeconds()+period));
        case 'minute':
            return new Date(date.setMinutes(date.getMinutes()+period));
        case 'hour':
            return new Date(date.setHours(date.getHours()+period));
        case 'day':
            return new Date(date.setDate(date.getDate()+period));
        case 'month':
            return new Date(date.setMonth(date.getMonth()+period));
        default:
            return undefined
    }
}

function sample_timestamps(start, end, period, period_unit) {
    var cDate = new Date(start);
    var sampled_dates = []
    while (cDate < end) {
        sampled_dates.push(cDate);
        cDate = increase(cDate, period, period_unit);
    }
    return sampled_dates;
}

function is_valid(smp) {
  // Check if the sample is valid (i.e. contains no clouds or snow)
  let clm = smp.CLM;
  let dm = smp.dataMask;

  if (clm === 1 || clm === 255) {
        return false;
  }
  if (dm !=1 ) {
        return false;
  }
  return true;
}

function withoutTime(intime) {
  // Return date without time
  intime.setHours(0, 0, 0, 0);
  return intime;
}

// Sentinel Hub functions
function setup() {
  // Setup input/output parameters
    return {
        input: [{
            bands: ["B04", "B08", "CLM", "dataMask"],
            units: "DN"
        }],
      output: [
          {id: "NDVI", bands: nb_bands, sampleType: SampleType.UINT16},
          {id: "data_mask", bands: nb_bands, sampleType: SampleType.UINT8}
      ],
    mosaicking: "ORBIT"
    }
}

// Evaluate pixels in the bands
function evaluatePixel(samples, scenes) {

  // Initialise arrays
  var valid_samples = {'B04':[], 'B08':[]};

  var valid_dates = []
  // Loop over samples.
  for (var i = samples.length-1; i >= 0; i--){
      if (is_valid(samples[i])) {
        valid_dates.push(withoutTime(new Date(scenes[i].date)));
        valid_samples['B04'].push(samples[i].B04);
        valid_samples['B08'].push(samples[i].B08);
      }
  }

  // Calculate indices and return optimised for UINT16 format (will need unpacking)
  var ndvi = interpolated_index(valid_samples['B08'], valid_samples['B04'])

  var [ndvi_interpolated, dm] = lininterp(sampled_dates, valid_dates, ndvi, 0);

  // Return all arrays
  return {
    NDVI: ndvi,
    data_mask: dm
  }
}"""

In [ ]:
time_interval = dt.date(year=2023, month=7, day=1), dt.date(year=2023, month=7, day=30)

Now we can define a Process API request and test it on a smaller sub-area to make sure we get back desired data.

In [ ]:
%%time

sentinelhub_request = SentinelHubRequest(
    evalscript=evalscript,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L1C,
            time_interval=time_interval,
        )
    ],
    responses=[
        SentinelHubRequest.output_response("NDVI", MimeType.TIFF),
        SentinelHubRequest.output_response("data_mask", MimeType.TIFF),
    ],
    bbox=test_bbox,
    size=bbox_to_dimensions(test_bbox, 10),
    config=config,
)

results = sentinelhub_request.get_data()[0]

print(f"Output data: {list(results)}")
plot_image(results["NDVI.tif"][..., 2])

We obtained stacks of NDVI values and data masks. 

### 1.2 Define a batch client

The interface for Sentinel Hub Batch API is class `SentinelHubBatch`. We initialize it with a configuration object that contains credentials and URLs of the services.

In [ ]:
batch = SentinelHubBatch(config=config)

### 1.3 Select a tiling grid

Batch API offers a number of pre-defined tiling grids. We can check which ones are available.

In [ ]:
list(batch.iter_tiling_grids())

Let's select a 10km grid, which is based on Sentinel-2 data tiling grid in UTM coordinate reference systems.

There is also an option to check a definition for a single grid:

In [ ]:
# Specify grid ID here:
GRID_ID = 1

batch.get_tiling_grid(GRID_ID)

### 1.4 Set up an S3 bucket

For this step please follow [instructions](https://docs.sentinel-hub.com/api/latest/api/batch/#aws-s3-bucket-settings) on how to configure an S3 bucket in a way that Sentinel Hub service will be able to write to it.

In [ ]:
# Write bucket name here:
BUCKET_NAME = ""

### 1.5 Join batch request definition

Now we are ready to create an entire batch request. This step won't trigger the actual processing. It will only save a batch request definition to the server-side.

In [ ]:
sentinelhub_request = SentinelHubRequest(
    evalscript=evalscript,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L1C,
            time_interval=time_interval,
        )
    ],
    responses=[
        SentinelHubRequest.output_response("NDVI", MimeType.TIFF),
        SentinelHubRequest.output_response("data_mask", MimeType.TIFF),
    ],
    geometry=full_geometry,
    # This time we don't specify size parameter
    config=config,
)

batch_request = batch.create(
    sentinelhub_request,
    tiling_grid=SentinelHubBatch.tiling_grid(grid_id=GRID_ID, resolution=10, buffer=(50, 50)),
    bucket_name=BUCKET_NAME,
    description="NDVI batch job",
)

batch_request

A batch request has been successfully created. The information about a request is provided in the form of a `BatchRequest` dataclass object. From the object representation, we can see some of its main properties, such as `status`, which defines the current status of a batch request. 

We can also check its full payload:

In [ ]:
batch_request.to_dict()

## 2. Analyse a batch request

Before we run a batch request job we can check currently defined batch requests and run an analysis to determine the outcome of a batch request. Important information we can obtain from this step are:

- the exact geometries of tiles from a tiling grid that will be processed,
- the number of processing units that a batch job will cost.

Note that this analysis paragraph is optional and is not required to run a batch request job.

### 2.1 Run an analysis

At the moment we don't have information about tiles or processing units yet. But we can order the service to calculate it.

The following will start the analysis on the server-side:

In [ ]:
batch.start_analysis(batch_request)

Depending on the size of our batch request it might take from a few seconds to a few minutes for analysis to finish. To determine if the analysis has finished we have to update batch request info and check the `status` information:

In [ ]:
batch_request = batch.get_request(batch_request)

batch_request

Once analysis is completed the `valueEstimate` tells us an estimated number of processing units the batch job will cost.

In [ ]:
print(f"Running this batch job will take about {batch_request.value_estimate:.4f} processing units")

### 2.2 Check tile definitions

When the analysis is complete we can check information about tiles:

In [ ]:
for tile_info in batch.iter_tiles(batch_request):
    print(tile_info)

Optionally, we can request information about a single tile:

In [ ]:
# Specify a tile ID
TILE_ID = ""

batch.get_tile(batch_request, TILE_ID)

To interact with tiles we can also use a type of an `AreaSplitter` class which already parses geometries:

In [ ]:
from sentinelhub import BatchSplitter

splitter = BatchSplitter(batch_request=batch_request, config=config)
splitter.get_bbox_list()

Let's plot the geometries:

In [ ]:
def plot_batch_splitter(splitter):
    """Plots tiles and area geometry from a splitter class"""
    tile_geometries = [Geometry(bbox.geometry, bbox.crs) for bbox in splitter.get_bbox_list()]
    tile_geometries = [geometry.transform(splitter.crs) for geometry in tile_geometries]

    gdf = gpd.GeoDataFrame(
        {"status": [info["status"] for info in splitter.get_info_list()]},
        geometry=[geometry.geometry for geometry in tile_geometries],
        crs=splitter.crs.pyproj_crs(),
    )
    gdf = gdf.dissolve(by="status").reset_index()
    color_map = {
        "PROCESSED": "tab:green",
        "FAILED": "tab:red",
        "PENDING": "tab:blue",
        "SCHEDULED": "tab:cyan",
    }

    _, ax = plt.subplots(figsize=(10, 10))
    pmarks = []

    for status, sdf in gdf.groupby("status"):
        sdf.plot(ax=ax, color=color_map[status], label=status)
        pmarks.append(Patch(facecolor=color_map[status], label=status))

    area_series = gpd.GeoSeries([splitter.get_area_shape()], crs=splitter.crs.pyproj_crs())
    area_series.plot(ax=ax, facecolor="none", edgecolor="black")

    handles, _ = ax.get_legend_handles_labels()
    ax.legend(handles=[*handles, *pmarks], loc="lower right")


plot_batch_splitter(splitter)

## 3. Run a batch request job

Once we decide to run a batch request job we can trigger it with the following:

In [ ]:
batch.start_job(batch_request)

Again we can check if a job has finished by updating batch request info.

In [ ]:
batch_request = batch.get_request(batch_request)

batch_request

This package also provides a utility function that monitors batch job execution by periodically checking for status of all tiles and sleeping in between.

In [ ]:
monitor_batch_job(batch_request, config=config, sleep_time=60)  # It will update progress every 60 seconds

Another option is to check which results have already been saved to the given S3 bucket.

When the job is running we can decide at any time to cancel it. Results that have already been produced will remain on the bucket.

In [ ]:
batch.cancel_job(batch_request)

When a job has finished we can check the status in batch request info and statuses of each tile:

In [ ]:
splitter = BatchSplitter(batch_request=batch_request, config=config)

plot_batch_splitter(splitter)

In case processing for any tile fails we have an option to re-run the job again. This will only run the processing for the tiles that failed.

In [ ]:
batch.restart_job(batch_request)

Alternatively, we can re-run processing only for a single tile:

In [ ]:
# Specify an ID of a tile that failed
FAILED_TILE_ID = ""

batch.reprocess_tile(batch_request, FAILED_TILE_ID)